## Count Total Votes for President Check

Check total votes for president
Check which ballots a candidate had zero votes

In [1]:
from pathlib import Path
import pandas as pd

BASE_DIR = Path(".").resolve().parent

bu_path = BASE_DIR / "bu"
ballot_metadata_path = BASE_DIR / "results_csv" / "ballot_metadata"
ballot_votes_path = BASE_DIR / "results_csv" / "ballot_votes"

[ i.name for i in bu_path.iterdir() if i.suffix == '.bu']


ballot_files = [ i.name for i in ballot_metadata_path.iterdir() if i.suffix == '.csv']
vote_files = [ i.name for i in ballot_votes_path.iterdir() if i.suffix == '.csv']

In [2]:
ballot_dfs = [ pd.read_csv(ballot_metadata_path / file) for file in ballot_files]
ballot_df = pd.concat(ballot_dfs)

ballot_dfs = [ pd.read_csv(ballot_votes_path / file) for file in vote_files]
votes_df = pd.concat(ballot_dfs)

In [3]:
ballot_df.shape

(464401, 6)

In [4]:
votes_df.shape

(2815247, 6)

In [5]:
votes_df.columns

Index(['ballot_id', 'position', 'vote_type', 'party_code', 'candidate_code',
       'qty_vote'],
      dtype='object')

In [6]:
votes_with_ballots = pd.merge(votes_df,ballot_df,how="left",on="ballot_id")

In [7]:
votes_with_ballots = votes_with_ballots[votes_with_ballots["position"]=="presidente"]

In [8]:
votes_with_ballots.shape

(1820846, 11)

In [9]:
votes_with_ballots_nominal = votes_with_ballots[votes_with_ballots["vote_type"]=="nominal"]

In [10]:
votes_with_ballots_nominal.groupby(["party_code"])[["qty_vote"]].sum()

,qty_vote
party_code,
13.0,59016829
22.0,57492231


In [11]:
lula_missing = 60_345_999 - 59_016_829 
print(f"{lula_missing:,d}")
bolsonaro_missing = 58_206_354 - 57_492_231
print(f"{bolsonaro_missing:,d}")

1,329,170
714,123


In [12]:
votes_with_ballots_nominal.sort_values(by="qty_vote")

,ballot_id,position,vote_type,party_code,candidate_code,qty_vote,state_code,council_name,file_name,zone,section_cd
1811822,1202500050200,presidente,nominal,22.0,22.0,1,PI,SÃO JOÃO DA VARJOTA,o00407-1202500050200.bu,5,200
424386,227500070046,presidente,nominal,22.0,22.0,1,AM,CODAJÁS,o00407-0227500070046.bu,7,46
2418929,3537801930116,presidente,nominal,22.0,22.0,1,BA,IAÇU,o00407-3537801930116.bu,193,116
2678967,863000450101,presidente,nominal,22.0,22.0,1,MA,PENALVA,o00407-0863000450101.bu,45,101
1804161,1040500740105,presidente,nominal,22.0,22.0,1,PI,PASSAGEM FRANCA DO PIAUÍ,o00407-1040500740105.bu,74,105
...,...,...,...,...,...,...,...,...,...,...,...
223729,2966100013108,presidente,nominal,13.0,13.0,512,ZZ,DUBLIN,o00407-2966100013108.bu,1,3108
225976,3045700013199,presidente,nominal,13.0,13.0,522,ZZ,AMSTERDÃ,o00407-3045700013199.bu,1,3199
225968,3045700013125,presidente,nominal,13.0,13.0,525,ZZ,AMSTERDÃ,o00407-3045700013125.bu,1,3125
224521,2938600011714,presidente,nominal,13.0,13.0,530,ZZ,BERLIM,o00407-2938600011714.bu,1,1714


In [13]:
# votes_with_ballots_nominal.to_csv("total_votes_president.csv", index=False)

In [23]:
ballot_count = votes_with_ballots_nominal.groupby("ballot_id")[["position"]].count()

In [27]:
single_vote_sessions = ballot_count[ballot_count["position"]==1]

In [29]:
votes_with_single_winner = pd.merge(votes_with_ballots_nominal,single_vote_sessions, on="ballot_id")

In [31]:
votes_with_single_winner["party_code"].value_counts()

13.0    141
22.0      4
Name: party_code, dtype: int64

In [32]:
votes_with_single_winner.to_csv("section_with_single_winner.csv", index=False)